<a href="https://colab.research.google.com/github/Vishwassaini20/final00009/blob/main/BROupdated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BRO

In [ ]:
import pandas as pd
import numpy as np
import folium
import os
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from geopy.geocoders import Nominatim
from google.colab import files
from collections import defaultdict
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from concurrent.futures import ThreadPoolExecutor
from sklearn.cluster import KMeans

# Constants
JECRC_LOCATION = (26.78218919094841, 75.82251239614644)  # JECRC College Lat/Lon
BUS_CAPACITY = 60
RESULTS_FOLDER = "results"
os.makedirs(RESULTS_FOLDER, exist_ok=True)

def haversine_distance(coord1, coord2):
    """Calculate the Haversine distance between two lat/lon coordinates."""
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Earth radius in km

    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lat2 - lon1)
    a = np.sin(dlat / 2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

def get_route(graph, start, end):
    """Get the route between start and end using OSMnx."""
    start_node = ox.distance.nearest_nodes(graph, start[1], start[0])
    end_node = ox.distance.nearest_nodes(graph, end[1], end[0])
    try:
        route = nx.shortest_path(graph, start_node, end_node, weight='length')
        route_coords = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in route]
        distance = nx.shortest_path_length(graph, start_node, end_node, weight='length') / 1000  # Convert to km
        return route_coords, distance
    except nx.NetworkXNoPath:
        print(f"No path found from {start} to {end}")
        return [], 0

def cluster_students(student_data, num_clusters):
    """Cluster students into groups using K-Means."""
    coords = np.array([[student[2], student[3]] for student in student_data])
    kmeans = KMeans(n_clusters=num_clusters).fit(coords)
    clusters = {i: [] for i in range(num_clusters)}
    for student, label in zip(student_data, kmeans.labels_):
        clusters[label].append(student)
    return clusters

def refine_clusters(graph, clusters, driver_locations):
    """Refine clusters to ensure optimal allocation based on shortest path distance."""
    refined_clusters = {bus: [] for bus in driver_locations}
    for cluster_id, students in clusters.items():
        for student in students:
            student_id, name, lat, lon = student
            student_coords = (lat, lon)
            closest_bus = None
            min_distance = float('inf')
            for bus, driver_loc in driver_locations.items():
                _, distance = get_route(graph, student_coords, driver_loc)
                if distance < min_distance:
                    min_distance = distance
                    closest_bus = bus
            refined_clusters[closest_bus].append(student)
    return refined_clusters

def solve_vrp(distance_matrix, fuel_cost_matrix, num_vehicles, depot=0):
    """Solves the Vehicle Routing Problem with fuel cost minimization."""
    size = len(distance_matrix)
    manager = pywrapcp.RoutingIndexManager(size, num_vehicles, depot)
    routing = pywrapcp.RoutingModel(manager)

    def cost_callback(from_index, to_index):
        return int(fuel_cost_matrix[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)])

    transit_callback_index = routing.RegisterTransitCallback(cost_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_parameters.time_limit.seconds = 30

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        routes = []
        for vehicle_id in range(num_vehicles):
            index = routing.Start(vehicle_id)
            route = []
            while not routing.IsEnd(index):
                route.append(manager.IndexToNode(index))
                index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
            routes.append(route)
        return routes
    return []

def create_map_and_image(bus_name, stops, route, students, graph):
    route_map = folium.Map(location=JECRC_LOCATION, zoom_start=12)
    folium.Marker(JECRC_LOCATION, popup='JECRC College', icon=folium.Icon(color='red')).add_to(route_map)

    # Track duplicate locations and apply small offsets
    location_counts = defaultdict(int)

    for student in students:
        student_id, name, lat, lon = student
        key = (lat, lon)  # Convert location to a hashable key
        location_counts[key] += 1

        # Apply a small offset if there are multiple students at the same location
        lat_offset = (location_counts[key] - 1) * 0.0001
        lon_offset = (location_counts[key] - 1) * 0.0001

        folium.Marker((lat + lat_offset, lon + lon_offset),
                      popup=f"{name} ({student_id})",
                      icon=folium.Icon(color='blue')).add_to(route_map)

    # Add polyline for the route
    for i in range(len(route) - 1):
        start = stops[route[i]]
        end = stops[route[i + 1]]
        route_coords, _ = get_route(graph, start, end)
        folium.PolyLine(route_coords, color='blue', weight=2.5, opacity=1).add_to(route_map)

    file_path_html = os.path.join(RESULTS_FOLDER, f"{bus_name}_route.html")
    route_map.save(file_path_html)
    files.download(file_path_html)

    # Generate PNG Image
    latitudes, longitudes = zip(*[stops[i] for i in route])
    plt.figure(figsize=(8, 6))
    plt.plot(longitudes, latitudes, marker='o', linestyle='-', color='b')
    plt.scatter(*zip(*stops), c='red', marker='x', label='Bus Stops')
    plt.scatter(*JECRC_LOCATION[::-1], c='green', marker='s', label='JECRC College')
    plt.legend()
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(f'Optimized {bus_name} Route')

    file_path_png = os.path.join(RESULTS_FOLDER, f"{bus_name}_route.png")
    plt.savefig(file_path_png)
    plt.close()
    files.download(file_path_png)

def generate_student_allocation_csv(bus_name, students):
    # Create DataFrame for student allocation
    allocation_data = []
    for student in students:
        student_id, name, lat, lon = student
        allocation_data.append([student_id, name, lat, lon])

    df = pd.DataFrame(allocation_data, columns=["Student ID", "Name", "Latitude", "Longitude"])
    file_path_csv = os.path.join(RESULTS_FOLDER, f"{bus_name}_student_allocation.csv")
    df.to_csv(file_path_csv, index=False)
    files.download(file_path_csv)

def calculate_fuel_cost(distance, fuel_price_per_liter, fuel_efficiency_km_per_liter):
    """Calculate the fuel cost based on distance, fuel price, and fuel efficiency."""
    return (distance / fuel_efficiency_km_per_liter) * fuel_price_per_liter

def main():
    global geolocator, students_df, buses_df
    geolocator = Nominatim(user_agent="bus_route_optimizer")
    graph = ox.graph_from_point(JECRC_LOCATION, dist=10000, network_type='drive')
    fuel_price_per_liter = float(input("Enter the fuel price per liter: "))
    fuel_efficiency_km_per_liter = float(input("Enter the fuel efficiency of the bus (km per liter): "))

    # Upload student data
    print("Please upload the students.csv file")
    uploaded_students = files.upload()
    students_df = pd.read_csv(list(uploaded_students.keys())[0])
    students_data = students_df.values.tolist()

    # Upload bus data
    print("Please upload the buses.csv file")
    uploaded_buses = files.upload()
    buses_df = pd.read_csv(list(uploaded_buses.keys())[0])
    driver_locations = {row['Bus Name']: (row['Latitude'], row['Longitude']) for index, row in buses_df.iterrows()}

    num_clusters = len(driver_locations)
    clusters = cluster_students(students_data, num_clusters)
    bus_assignments = refine_clusters(graph, clusters, driver_locations)

    with ThreadPoolExecutor() as executor:
        futures = []
        for bus_name, students in bus_assignments.items():
            if students:
                futures.append(executor.submit(process_bus_route, bus_name, students, fuel_price_per_liter, fuel_efficiency_km_per_liter, graph))
        for future in futures:
            future.result()

def process_bus_route(bus_name, students, fuel_price_per_liter, fuel_efficiency_km_per_liter, graph):
    assigned_stops = np.array([driver_locations[bus_name]] + [s[2:4] for s in students] + [JECRC_LOCATION])
    distance_matrix = np.zeros((len(assigned_stops), len(assigned_stops)))
    fuel_cost_matrix = np.zeros((len(assigned_stops), len(assigned_stops)))

    for i, start in enumerate(assigned_stops):
        for j, end in enumerate(assigned_stops):
            if i != j:
                _, distance = get_route(graph, start, end)
                distance_matrix[i, j] = distance
                fuel_cost_matrix[i, j] = calculate_fuel_cost(distance, fuel_price_per_liter, fuel_efficiency_km_per_liter)

    num_vehicles = 1
    routes = solve_vrp(distance_matrix, fuel_cost_matrix, num_vehicles)
    for route in routes:
        route_coords = [assigned_stops[i] for i in route]
        create_map_and_image(bus_name, assigned_stops, route, students, graph)

        total_distance = sum(distance_matrix[route[i], route[i + 1]] for i in range(len(route) - 1))
        total_fuel_cost = sum(fuel_cost_matrix[route[i], route[i + 1]] for i in range(len(route) - 1))
        print(f"Total distance for {bus_name}: {total_distance:.2f} km")
        print(f"Total fuel cost for {bus_name}: ${total_fuel_cost:.2f}")

    generate_student_allocation_csv(bus_name, students)

if __name__ == "__main__":
    main()

#BRO WITH MULTI STOPS

In [ ]:
import pandas as pd
import numpy as np
import folium
import os
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from ortools.constraint_solver import routing_enums_pb2, pywrapcp
from google.colab import files
from collections import defaultdict

# Constants
JECRC_LOCATION = (26.78218919094841, 75.82251239614644)  # JECRC College Lat/Lon
BUS_CAPACITY = 60
RESULTS_FOLDER = "results"
os.makedirs(RESULTS_FOLDER, exist_ok=True)

# Step 1: Upload Students Data
print("Upload students_data.csv file")
uploaded_students = files.upload()
students_df = pd.read_csv("students_data.csv")
students_data = students_df.values.tolist()

# Step 2: Upload Bus Routes Data
print("Upload bus_routes.csv file")
uploaded_buses = files.upload()
bus_routes_df = pd.read_csv("bus_routes.csv")

# Convert bus routes CSV into a dictionary with tuple (lat, long)
bus_routes = {}
for _, row in bus_routes_df.iterrows():
    bus_name = row.iloc[0]  # First column is the bus name
    stops =[(float(latlon.split(',')[0]), float(latlon.split(',')[1])) for latlon in bus_data.iloc[:, 1:].values[0] if isinstance(latlon, str)] # Convert string tuple to actual tuple
    bus_routes[bus_name] = stops


def assign_students_to_buses(student_data, bus_routes):
    assignments = {bus: [] for bus in bus_routes}

    for student in student_data:
        _, name, lat, lon = student
        nearest_bus, nearest_stop, min_distance = None, None, float('inf')

        for bus, stops in bus_routes.items():
            for stop in stops:
                distance = np.linalg.norm(np.array([lat, lon]) - np.array(stop))
                if distance < min_distance and len(assignments[bus]) < BUS_CAPACITY:
                    min_distance = distance
                    nearest_bus, nearest_stop = bus, stop

        if nearest_bus:
            assignments[nearest_bus].append((student[0], name, nearest_stop))

    return assignments

def solve_tsp(distance_matrix):
    """ Solves TSP ensuring the first node is JECRC College and the last node is the last stop. """
    size = len(distance_matrix)
    manager = pywrapcp.RoutingIndexManager(size, 1, [0], [size - 1])  # Start at JECRC (index 0), end at last stop
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        return int(distance_matrix[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        route = []
        index = routing.Start(0)
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        return route  # Last stop is now the final destination
    return []


def create_map_and_image(bus_name, stops, route, students):
    route_map = folium.Map(location=JECRC_LOCATION, zoom_start=12)
    folium.Marker(JECRC_LOCATION, popup='JECRC College', icon=folium.Icon(color='red')).add_to(route_map)

    # Track duplicate locations and apply small offsets
    location_counts = defaultdict(int)

    for student_id, name, stop in students:
        key = tuple(stop)  # Convert location to a hashable key
        location_counts[key] += 1
        lat, lon = stop

        # Apply a small offset if there are multiple students at the same location
        lat_offset = (location_counts[key] - 1) * 0.0001
        lon_offset = (location_counts[key] - 1) * 0.0001

        folium.Marker((lat + lat_offset, lon + lon_offset),
                      popup=f"{name} ({student_id})",
                      icon=folium.Icon(color='blue')).add_to(route_map)

    # Ensure JECRC is the first and last stop
    route_stops = [JECRC_LOCATION] + [stops[i] for i in route if i < len(stops)] + [JECRC_LOCATION]

    folium.PolyLine(route_stops, color='blue', weight=2.5, opacity=1).add_to(route_map)

    file_path_html = os.path.join(RESULTS_FOLDER, f"{bus_name}_route.html")
    route_map.save(file_path_html)
    files.download(file_path_html)

    # Generate PNG Image
    latitudes, longitudes = zip(*route_stops)
    plt.figure(figsize=(8, 6))
    plt.plot(longitudes, latitudes, marker='o', linestyle='-', color='b')
    plt.scatter(*zip(*stops), c='red', marker='x', label='Bus Stops')
    plt.scatter(*JECRC_LOCATION[::-1], c='green', marker='s', label='JECRC College')
    plt.legend()
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(f'Optimized {bus_name} Route')

    file_path_png = os.path.join(RESULTS_FOLDER, f"{bus_name}_route.png")
    plt.savefig(file_path_png)
    plt.close()
    files.download(file_path_png)

def generate_student_allocation_csv(bus_name, students):
    # Create DataFrame for student allocation
    allocation_data = []
    for student_id, name, stop in students:
        allocation_data.append([student_id, name, stop[0], stop[1]])

    df = pd.DataFrame(allocation_data, columns=["Student ID", "Name", "Latitude", "Longitude"])
    file_path_csv = os.path.join(RESULTS_FOLDER, f"{bus_name}_student_allocation.csv")
    df.to_csv(file_path_csv, index=False)
    files.download(file_path_csv)

def main():
    assignments = assign_students_to_buses(students_data, bus_routes)

    for bus_name, students in assignments.items():
        if not students:
            continue

        # Include JECRC College as a start and end point
        assigned_stops = np.array([JECRC_LOCATION] + [s[2] for s in students] + [JECRC_LOCATION])
        distance_matrix = np.linalg.norm(assigned_stops[:, None] - assigned_stops, axis=-1)

        route = solve_tsp(distance_matrix)
        create_map_and_image(bus_name, assigned_stops, route, students)
        generate_student_allocation_csv(bus_name, students)

if __name__ == "__main__":
    main()